## Basketball Team - Modelling

#### José María Benítez, Pablo Berástegui, Daniel Escobosa y Alejandro de Haro

### Importing packages and modules

In [1]:
import pyomo.environ as pe
import pyomo.opt as po

### Create the model

In [2]:
model = pe.ConcreteModel()

#### Sets

$p$: player {'1', '2', '3', '4', '5', '6', '7', '8', '9'}

$pos$: position {'G', 'F', 'C'}

In [3]:
model.P = pe.Set(initialize=[1,2,3,4,5,6,7,8,9])
model.Pos = pe.Set(initialize=['G','F','C'])

#### Parameters

$M_{p,pos}$: the player $p$ plays as $pos$ position 

$H$: handle skill ranked depending on the player $p$

$S$: shot skill ranked depending on the player $p$

$RB$: rebound skill ranked depending on the player $p$

$D$: defense skill ranked depending on the player $p$

In [4]:
H = {1:2, 2:3, 3:2, 4:1, 5:1, 6:3, 7:3, 8:2, 9:3}
S = {1:1, 2:3, 3:3, 4:3, 5:3, 6:1, 7:2, 8:1, 9:3}
RB= {1:3, 2:1, 3:2, 4:3, 5:1, 6:2, 7:2, 8:3, 9:1}
D = {1:3, 2:2, 3:2, 4:1, 5:2, 6:3, 7:1, 8:2, 9:3}

model.H = pe.Param(model.P, initialize=H) #handle
model.S = pe.Param(model.P, initialize=S) #shot
model.RB = pe.Param(model.P, initialize=RB) #rebound
model.D = pe.Param(model.P, initialize=D) #defense

orden_pos = ['G','F','C']

pos_by_player = {
    1:{'C'}, 2:{'G'}, 3:{'F','C'}, 4:{'G','F'}, 5:{'F','C'},
    6:{'G','F'}, 7:{'F','C'}, 8:{'C'}, 9:{'F'}
}
pos_players = {(p,r): int(r in pos_by_player[p]) for p in model.P for r in model.Pos}
model.pos_players = pe.Param(model.P, model.Pos, initialize=pos_players)



#### Variables

$x_{p,pos}$: the player $p$ is selected to play as the position $pos$

In [5]:
model.x = pe.Var(model.P, model.Pos, within=pe.Binary)

#### Objective Function

**max $\sum_{p,pos}$ $x_{p,pos}$ $*$ $D_{p}$**

In [6]:
def obj_rule(model):
    return sum(model.D[p]*model.x[p,pos] for p in model.P for pos in model.Pos)
model.total_cost = pe.Objective(rule=obj_rule, sense=pe.maximize)

#### Constraints

### Restricciones del modelo de baloncesto

**1. Tamaño del equipo**

$$
\sum_{p \in P} \sum_{r \in R} x_{p,r} = 5
$$


**2. Requisitos de posiciones**

$$
\sum_{p \in P} x_{p,G} \;\geq\; 2
$$

$$
\sum_{p \in P} x_{p,F} \;\geq\; 2
$$

$$
\sum_{p \in P} x_{p,C} \;\geq\; 1
$$


**3. Promedios mínimos de habilidades**

(Como se seleccionan 5 jugadores, la suma debe ser al menos 10 en cada habilidad)

$$
\sum_{p \in P} \sum_{r \in R} H_p \, x_{p,r} \;\geq\; 10
$$

$$
\sum_{p \in P} \sum_{r \in R} S_p \, x_{p,r} \;\geq\; 10
$$

$$
\sum_{p \in P} \sum_{r \in R} RB_p \, x_{p,r} \;\geq\; 10
$$


**4. Incompatibilidad jugadores 3 y 6**

$$
\sum_{r \in R} x_{3,r} + \sum_{r \in R} x_{6,r} \;\leq\; 1
$$


**5. Relación entre jugadores 1, 4 y 5**

$$
\sum_{r \in R} x_{4,r} + \sum_{r \in R} x_{5,r} \;\geq\; \sum_{r \in R} x_{1,r}
$$

$$
\sum_{r \in R} x_{4,r} + \sum_{r \in R} x_{5,r} \;\leq\; 2 - \sum_{r \in R} x_{1,r}
$$


**6. Exclusión entre jugadores 8 y 9**

$$
\sum_{r \in R} x_{8,r} + \sum_{r \in R} x_{9,r} \;=\; 1
$$


In [7]:
def position_availability_rule(m,p,pos):
    return m.x[p,pos] <= m.pos_players[p,pos]
model.position_availability = pe.Constraint(model.P, model.Pos, rule=position_availability_rule)

# A lo sumo una posición por jugador
def atmost_one_pos_rule(m,p):
    return sum(m.x[p,pos] for pos in m.Pos) <= 1
model.atmost_one_pos = pe.Constraint(model.P, rule=atmost_one_pos_rule)

model.team_size = pe.Constraint(expr=sum(model.x[p,pos] for p in model.P for pos in model.Pos) == 5)

model.min_G = pe.Constraint(expr=sum(model.x[p,'G'] for p in model.P) >= 2)
model.min_F = pe.Constraint(expr=sum(model.x[p,'F'] for p in model.P) >= 2)
model.min_C = pe.Constraint(expr=sum(model.x[p,'C'] for p in model.P) >= 1)

model.min_H  = pe.Constraint(expr=sum(model.H[p]*sum(model.x[p,pos] for pos in model.Pos) for p in model.P) >= 10)
model.min_S  = pe.Constraint(expr=sum(model.S[p]*sum(model.x[p,pos] for pos in model.Pos) for p in model.P) >= 10)
model.min_RB = pe.Constraint(expr=sum(model.RB[p]*sum(model.x[p,pos] for pos in model.Pos) for p in model.P) >= 10)

model.incomp_36 = pe.Constraint(expr=sum(model.x[3,pos] for pos in model.Pos) + sum(model.x[6,pos] for pos in model.Pos) <= 1)

model.one_45_lb = pe.Constraint(expr=sum(model.x[4,pos] for pos in model.Pos) + sum(model.x[5,pos] for pos in model.Pos) >= sum(model.x[1,pos] for pos in model.Pos))
model.one_45_ub = pe.Constraint(expr=sum(model.x[4,pos] for pos in model.Pos) + sum(model.x[5,pos] for pos in model.Pos) <= 2 - sum(model.x[1,pos] for pos in model.Pos))

model.xor_89 = pe.Constraint(expr=sum(model.x[8,pos] for pos in model.Pos) + sum(model.x[9,pos] for pos in model.Pos) == 1)

#### Solver definition and solve statement

In [8]:
solver = po.SolverFactory('gurobi')
solver.solve(model, tee=False)

{'Problem': [{'Name': 'x1', 'Lower bound': 12.0, 'Upper bound': 12.0, 'Number of objectives': 1, 'Number of constraints': 47, 'Number of variables': 27, 'Number of binary variables': 27, 'Number of integer variables': 27, 'Number of continuous variables': 0, 'Number of nonzeros': 219, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.03900003433227539', 'Error rc': 0, 'Time': 1.0331964492797852}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

#### Results

In [15]:
selected = [(p, [pos for pos in model.Pos if pe.value(model.x[p,pos])>0.5][0])
            for p in model.P if sum(pe.value(model.x[p,pos]) for pos in model.Pos)>0.5]
defense = 0
for jug,pos in selected:
    defense+=D[jug]
print(f'Defensa: {defense}')
print(f"5 jugadores (jugador, posición): {selected}")

Defensa: 12
5 jugadores (jugador, posición): [(1, 'C'), (2, 'G'), (4, 'F'), (6, 'G'), (9, 'F')]
